In [16]:
# 目的是跑分，需要把 llama predict 出来的算个分

import os
import json

model_name = "codellama-34b-python"

valid_path = "output/hf-eval-data-v4-valid"
folder_path = f"output/hf-eval-data-v4-valid-result/{model_name}"
eval_path = f"output/hf-eval-data-v4-valid-result/{model_name}-eval"

try:
    os.mkdir(eval_path)
except:
    pass

def iter_files(folder_path):
    # 遍历文件列表
    for file in os.listdir(folder_path):
        # 获取文件的完整路径
        file_path = os.path.join(folder_path, file)

        # 判断是否为文件（而不是文件夹）
        if os.path.isfile(file_path):
            yield file_path

In [17]:
all_predict_result = []

for file in iter_files(folder_path):
    with open(file) as f:
        try:
            d = json.loads(f.read())
            all_predict_result.append(d)
        except:
            pass
    
len(all_predict_result), all_predict_result[0]

(49,
 {'path': 'output/hf-eval-data-v4-valid/f00055_translate_french_to_english.py',
  'content': '# requirements_file --------------------\n\nimport subprocess\n\nrequirements = ["transformers"]\n\nfor package in requirements:\n    subprocess.run([\'pip\', \'install\', \'-U\', package])\n\n# function_import --------------------\n\nfrom transformers import pipeline\n\n# function_code --------------------\n\ndef translate_french_to_english(text):\n    """\n    Translates a given text from French to English using Hugging Face\'s Transformers library.\n\n    Args:\n        text (str): The text in French to be translated.\n\n    Returns:\n        str: The translated text in English.\n\n    Raises:\n        ValueError: If the input text is not a string or if it\'s empty.\n    """\n    if not isinstance(text, str) or not text:\n        raise ValueError(\'Input text must be a non-empty string.\')\n    \n    translation_pipeline = pipeline(\'translation_fr_to_en\', model=\'Helsinki-NLP/opus-mt

In [18]:
# pip file + predict file => eval file
import shutil

try:
    os.mkdir(eval_path)
except:
    pass

for d in all_predict_result:
    print(d['path'])
    
    predict_file = d['path']
    
    # pip file
    pip_file = predict_file.split(".")[0] + ".txt"
    file_name = pip_file.split("/")[-1]
    
    try:
        shutil.copy(pip_file, eval_path + "/" + file_name)
    except:
        pass
    
    # eval python file
    eval_file = d['instruct'] + d['prediction'].split("\ndef")[0] + "\n\n" + d['test_function_code'] + "\n" + d['call_test_function_line']
    
    eval_file_path = eval_path + "/" + predict_file.split("/")[-1]
    
    print(eval_file_path)
    
    with open(eval_file_path, 'w') as fw:
        fw.write(eval_file)

output/hf-eval-data-v4-valid/f00055_translate_french_to_english.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00055_translate_french_to_english.py
output/hf-eval-data-v4-valid/f00064_generate_story_start.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00064_generate_story_start.py
output/hf-eval-data-v4-valid/f00065_predict_missing_text.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00065_predict_missing_text.py
output/hf-eval-data-v4-valid/f00066_fill_in_the_blank_chinese_sentence.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00066_fill_in_the_blank_chinese_sentence.py
output/hf-eval-data-v4-valid/f00070_calculate_sentence_similarity.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00070_calculate_sentence_similarity.py
output/hf-eval-data-v4-valid/f00085_predict_vehicle_co2_emissions.py
output/hf-eval-data-v4-valid-result/codellama-34b-python-eval/f00085_predict_vehicle_co2_emissions.py
outp

In [20]:
for idx, d in enumerate(hf_eval_data):
    print(idx + 1, end="...")
    formatted_number = str(idx + 1).zfill(5)

    # check err
    matching_files_err = glob.glob(f"{output_dir}/f{formatted_number}_*.err")
    if not matching_files_err:
        continue
    
    matched += 1
        
    with open(matching_files_err[0]) as f:
        content = f.read()
        if 'Error' in content:
            continue

    # check out
    matching_files_out = glob.glob(f"{output_dir}/f{formatted_number}_*.out")
    if not matching_files_out:
        continue
        
    with open(matching_files_out[0]) as f:
        content = f.read()
        if 'failed' in content:
            continue
            
    # copy files
    matching_files = glob.glob(f"{output_dir}/f{formatted_number}_*")
    for source_file in matching_files:
        dest_file = valid_dir + source_file.split(f"{output_dir}")[-1]
        shutil.copy(source_file, dest_file)
        
    valid += 1
    
valid, matched, valid/matched

NameError: name 'hf_eval_data' is not defined

In [22]:
# get eval pass rate
import os

for i in [7, 13, 34]:
    eval_dir = f"output/hf-eval-data-v3-reuslt-{i}b-eval/"
    total_case = 0
    pass_case = 0

    for file in os.listdir(eval_dir):
        if file.endswith(".err"):
            # print(file)

            total_case += 1

            # check err file
            err_file = file
            with open(eval_dir + err_file) as f:
                content = f.read()
                if 'Error' in content:
                    continue

            # check out file
            out_file = file.split(".")[0] + ".out"
            with open(eval_dir + out_file) as f:
                content = f.read()
                if 'failed' in content:
                    continue
            
            # print(file)
            pass_case += 1

    print(f"{i}b", pass_case, total_case, pass_case / total_case)

7b 44 241 0.1825726141078838
13b 51 239 0.21338912133891214
34b 51 241 0.21161825726141079
